In [ ]:
from google.colab import drive
drive.flush_and_unmount()
from google.colab import drive
drive.mount('/content/drive')


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import csv
warnings.filterwarnings("ignore")
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
import random
import math
from collections import Counter
from sklearn.metrics import confusion_matrix, accuracy_score,ConfusionMatrixDisplay, precision_recall_curve
from sklearn.metrics import classification_report, f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [ ]:
Allpath='/content/drive/My Drive/Colab Notebooks/Naghibzadeh/'
DataNum=4
if DataNum==1:
  DBname='df_final_NR.csv'
elif DataNum==2:
    DBname='df_final_IC.csv'
elif DataNum==3:
    DBname='df_final_GPCR.csv'
elif DataNum==4:
    DBname='df_final_EN.csv'
NewAdd=Allpath+'NewResult/'+DBname[:-4]+'/'



In [ ]:
nPCA=100
PCAFlag=1
NormFlag=1
scaler = MinMaxScaler()
# *** groups ***
# Read drug-target interactions
df_inter = pd.read_csv(Allpath+DBname)


df_inter.drop(df_inter.columns[df_inter.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)
#print(df_inter.head())

count_label_0 = sum(df_inter['label'] == 0)
count_label_1 = sum(df_inter['label'] == 1)


# **Splitting samples of label(0) **
df_label_0 = df_inter.loc[df_inter['label'] == 0]
df_label_1 = df_inter.loc[df_inter['label'] == 1]
hd1x = df_label_1.copy()
hd1y=df_label_1['label']
hd1x.drop(['drug_no', 'protein_no', 'label'], axis=1, inplace=True)
#print(hd1y)

hd0x = df_label_0.copy()
hd0y=df_label_0['label']
hd0x.drop(['drug_no', 'protein_no', 'label'], axis=1, inplace=True)


#print(hd0y)

hd1x = hd1x.to_numpy()
hd1y = hd1y.to_numpy()
hd0x = hd0x.to_numpy()
hd0y = hd0y.to_numpy()
if PCAFlag==1:
  XPCANew=np.concatenate([hd0x,hd1x])
  pca = PCA(n_components=nPCA)
  pca.fit(XPCANew)
  XPCANew=pca.transform(XPCANew)
  hd0x=XPCANew[:len(hd0x),:]
  hd1x=XPCANew[len(hd0x):,:]


rand0 = np.random.permutation(hd0x.shape[0])
rand1 = np.random.permutation(hd1x.shape[0])
index0=math.ceil(0.25*hd0x.shape[0])
index1=math.ceil(0.25*hd1x.shape[0])

hd0x_test=hd0x[rand0[:index0],:]
hd0x=hd0x[rand0[index0+1:],:]
hd0y_test = hd0y[rand0[:index0]]
hd0y=hd0y[rand0[index0+1:]]

hd1x_test=hd1x[rand1[:index1],:]
hd1x=hd1x[rand1[index1:],:]
hd1y_test = hd1y[rand1[:index1]]
hd1y=hd1y[rand1[index1:]]



XPCA=np.concatenate([hd0x,hd1x])

if NormFlag==1:
  scaler.fit(XPCA)
  XPCA=scaler.transform(XPCA)
hd0x=XPCA[:len(hd0x),:]
hd1x=XPCA[len(hd0x):,:]

XPCA_test=np.concatenate([hd0x_test,hd1x_test])

if NormFlag==1:
  XPCA_test=scaler.transform(XPCA_test)
hd0x_test=XPCA_test[:len(hd0x_test),:]
hd1x_test=XPCA_test[len(hd0x_test):,:]





In [ ]:
Xnew=np.concatenate([hd0x,hd1x,hd0x_test,hd1x_test])
Ynew=np.concatenate([hd0y,hd1y,hd0y_test,hd1y_test])
XAll=np.concatenate([Xnew,Ynew.reshape(-1,1)],axis=1)
hd=pd.DataFrame(data=XAll)
#hd.head()
print('Sample: ', hd.shape[0])
print('attributes: ', hd.shape[1] - 1)
hd.describe()


def draw_histograms(dataframe, features, rows, cols):
    num_features = len(features)
    max_plots = rows * cols 
    num_pages = (num_features + max_plots - 1) // max_plots  

    for page in range(num_pages):
        start_idx = page * max_plots
        end_idx = min(start_idx + max_plots, num_features)  
        fig = plt.figure(figsize=(20, 20))

        for i, feature in enumerate(features[start_idx:end_idx]):
            ax = fig.add_subplot(rows, cols, i + 1)
            dataframe[feature].hist(bins=20, ax=ax, facecolor='DarkBlue')
            ax.set_title(feature, color='DarkRed')

        fig.tight_layout()
        plt.show()


if 15 not in hd.columns:
    raise KeyError("Column 15 does not exist in the DataFrame. Please check your column indices.")


num_features = len(hd.columns)
max_features = 18  


draw_histograms(hd, hd.columns[:min(num_features, max_features)], rows=6, cols=3)


numeric_features = hd.select_dtypes(include=[np.number])
corr = numeric_features.corr(method='spearman')
plt.figure(figsize=(max(10, corr.shape[1] * 1.5), max(10, corr.shape[0] * 1.5)))
sns.heatmap(corr, square=True, cbar=True, annot=True, fmt=".2f")
plt.title("Spearman Correlation Heatmap")
plt.show()


chdc = Counter(hd[15])
print('Interaction: ', chdc[1])
print('Non-Interaction: ', chdc[0])
plt.bar(list(chdc.keys()), list(chdc.values()), color=['g', 'r'])


print(f"Unique values in column 15: {list(chdc.keys())}")


if len(chdc.keys()) == 2:  
    plt.bar(list(chdc.keys()), list(chdc.values()), color=['g', 'r'])
    plt.xticks(list(chdc.keys()), ['No', 'Yes'])
    plt.title('Interaction and Non-Interaction')
    plt.show()


plt.title('Interaction and Non-Interaction')
plt.show()




Output hidden; open in https://colab.research.google.com to view.

# New Section

In [ ]:
X=np.concatenate([hd0x,hd1x,hd0x_test,hd1x_test])
Y=np.concatenate([hd0y,hd1y,hd0y_test,hd1y_test])
from sklearn.model_selection import train_test_split
newtrain_X, newtest_X, newtrain_Y, newtest_Y = train_test_split(X, Y, test_size = 0.2, random_state = 42)

X_rsam=np.concatenate([newtrain_X,newtest_X])
Y_rsam=np.concatenate([newtrain_Y,newtest_Y])
cc = Counter(newtest_Y)
print(cc[1])
print(cc[0])

print(newtrain_X.shape)
print(newtest_X.shape)
print(newtrain_Y.shape)
print(newtest_Y.shape)

print(newtrain_Y[10:20])

575
596
(4680, 100)
(1171, 100)
(4680,)
(1171,)
[0 0 1 1 1 1 1 0 0 0]


In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
from imblearn.under_sampling import AllKNN
from imblearn.under_sampling import InstanceHardnessThreshold
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTEN
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
OLD_reSampler=[[ClusterCentroids(estimator=MiniBatchKMeans(n_init=1, random_state=0), random_state=42),
                    CondensedNearestNeighbour(random_state=42),
                    EditedNearestNeighbours(),
                    RepeatedEditedNearestNeighbours(),
                    #AllKNN(),
                    #InstanceHardnessThreshold(random_state=42),
                    NeighbourhoodCleaningRule(),
                    OneSidedSelection(random_state=42),
                    RandomUnderSampler(random_state=42),
                    TomekLinks(),
                    ],[ RandomOverSampler(random_state=42),
                    SMOTE(random_state=42),
                    SMOTENC(random_state=42, categorical_features=[18, 19]),
                    SMOTEN(random_state=0),
                    #ADASYN(random_state=42),
                    BorderlineSMOTE(random_state=42),
#                    KMeansSMOTE(kmeans_estimator=MiniBatchKMeans(n_init=1, random_state=0), random_state=42),
                    SVMSMOTE(random_state=42)
                    ], [#SMOTEENN(random_state=42),
                  SMOTETomek(random_state=42),
                  ]]

OLD_reSamplerName=[['ClusterCentroids',
                    'ConNearNeigh',
                    'EditNearNeighb',
                    'ReEditNearNeighb',
                    #'AllKNN',
                    #'InstHardThre',
                    'NeighbCleanRule',
                    'OneSidedSel',
                    'RandUnderSam',
                    'TomekLinks',
                    ],[ 'RandOverSampler',
                    'SMOTE',
                    'SMOTENC',
                    'SMOTEN',
                    #'ADASYN',
                    'BorderlineSMOTE',
                    #'KMeansSMOTE',
                    'SVMSMOTE'
                    ], [#'SMOTEENN',
                  'SMOTETomek',
                  ]]
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import RUSBoostClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from numpy import array, concatenate, mean, std, reshape, zeros, ones, arange
import numpy as np
names = [
    "Nearest Neighbors",
    "Linear SVM",
    #"RBF SVM",
    #"Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
    "LDA",
    "EEC",
    "RSC",
    "BBC",
    "BRFC"

]
base_model11 = DecisionTreeClassifier(max_depth=5)

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=10),
    #SVC(gamma=2, C=1, random_state=42,probability=True),
    #GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
    DecisionTreeClassifier(max_depth=5, random_state=42),
    RandomForestClassifier(
        max_depth=50, n_estimators=100),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    AdaBoostClassifier(n_estimators=10, random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    LinearDiscriminantAnalysis(),
    EasyEnsembleClassifier(random_state=42),
    RUSBoostClassifier(n_estimators=10),  # Changed n_estimatorsrandom_state=0),
    BalancedBaggingClassifier(random_state=42),
    BalancedRandomForestClassifier(sampling_strategy="all", replacement=True, max_depth=2, random_state=0, bootstrap=False)
]
def evaluate_model(clf, X, y):
    pred = clf.predict(X) #predicted classes
    accuracy = accuracy_score(pred,y) # calculate accuracy
    fpr, tpr, _ = roc_curve(y, clf.predict(X)) # roc_curve
    auc_value = auc(fpr,tpr) # auc_value
    report = classification_report(y, pred, labels=[0,1], output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df = report_df.reset_index()
    model_eval  = report_df[report_df['index'].str.contains('1')][['precision','recall','f1-score']]
    model_eval['accuracy']  = list(report_df[report_df['index'].str.contains('accuracy')]['support'])
    model_eval['ROC']  = auc_value
    cf_matrix = confusion_matrix(y, pred)
    return model_eval,pred,y, cf_matrix

def model_eval_data(clf, X_train, y_train,
                         X_test, y_test,
                         model_eval_train,
                         model_eval_test,
                         model_eval_train_y,
                         model_eval_test_y,
                         SaveStep1,
                         Name=None):

    temp_eval_train,pred_train1,y_train1, cf_matrix_train = evaluate_model(clf, X_train, y_train)
    temp_eval_test,pred_test1,y_test1, cf_matrix_test = evaluate_model(clf, X_test, y_test)

    temp_train_y=pd.DataFrame()
    temp_test_y=pd.DataFrame()
    if SaveStep1==1:
      temp_train_y['Value']=[y_train1]
      temp_test_y['Value']=[y_test1]
      temp_train_y.index = ['real']
      temp_test_y.index = ['real']
      try:
        model_eval_train_y = pd.concat([model_eval_train_y, temp_train_y])
        model_eval_test_y = pd.concat([model_eval_test_y, temp_test_y])
        #model_eval_train = model_eval_train.append(temp_eval_train)
        #model_eval_test = model_eval_test.append(temp_eval_test)
      except:
        model_eval_train_y = temp_train_y
        model_eval_test_y = temp_test_y

    temp_eval_train.index = [Name]
    temp_eval_test.index = [Name]
    temp_train_y['Value']=[pred_train1]
    temp_test_y['Value']=[pred_test1]
    temp_train_y.index = [Name]
    temp_test_y.index = [Name]

    try:
        model_eval_train = pd.concat([model_eval_train, temp_eval_train])
        model_eval_test = pd.concat([model_eval_test, temp_eval_test])
        model_eval_train_y = pd.concat([model_eval_train_y, temp_train_y])
        model_eval_test_y = pd.concat([model_eval_test_y, temp_test_y])
        #model_eval_train = model_eval_train.append(temp_eval_train)
        #model_eval_test = model_eval_test.append(temp_eval_test)
    except:
        model_eval_train = temp_eval_train
        model_eval_test = temp_eval_test
        model_eval_train_y = temp_train_y
        model_eval_test_y = temp_test_y

    return model_eval_train, model_eval_test,model_eval_train_y, model_eval_test_y, cf_matrix_train, cf_matrix_test
def make_confusion_matrix_chart(cf_matrix_train, cf_matrix_test,pltname):

    #plt.figure()
    #plt.title(pltname)
    #plt.subplot(121)
    disp = ConfusionMatrixDisplay(confusion_matrix=cf_matrix_train,
                              display_labels=['Not Interaction', 'Interaction'])

    disp.plot()
    plt.xticks(rotation='horizontal')
    plt.yticks(rotation='vertical')
    plt.title('Train_'+pltname)
    #plt.gca().set_xticklabels([])
    plt.savefig(NewAdd+'Train_'+pltname+'.png')

    disp = ConfusionMatrixDisplay(confusion_matrix=cf_matrix_test,
                              display_labels=['Not Interaction', 'Interaction'])

    disp.plot()
    plt.xticks(rotation='horizontal')
    plt.yticks(rotation='vertical')
    plt.title('Test_'+pltname)
    #plt.gca().set_xticklabels([])
    plt.savefig(NewAdd+'Test_'+pltname+'.png')

    return None





In [ ]:
saveStep1=1
model_eval_train = pd.DataFrame({},[])
model_eval_test = pd.DataFrame({},[])
model_eval_train_y = pd.DataFrame({},[])
model_eval_test_y = pd.DataFrame({},[])
for name_res1, clf_res1 in zip(OLD_reSamplerName, OLD_reSampler):
  for name_res2, clf_res2 in zip(name_res1, clf_res1):
    print('Resample: ',name_res2)
    X_augmented_data, y_augmented_data = clf_res2.fit_resample(X_rsam, Y_rsam)
    for name, clf in zip(names, classifiers):
      print(name_res2+'-'+name+'-')
      '''print(X_augmented_data.shape)
      count_ones = np.count_nonzero(y_augmented_data == 1)
      count_zeros = np.count_nonzero(y_augmented_data == 0)
      print("Number of 1s:", count_ones)
      print("Number of 0s:", count_zeros)'''
      #start_time = time.time()
      clf1 = clf.fit(X_augmented_data, y_augmented_data)
      #clf1 =clf.fit(newtest_X, newtest_Y)
      model_eval_train, model_eval_test,model_eval_train_y,model_eval_test_y, cf_matrix_train, cf_matrix_test = model_eval_data(clf1, newtrain_X, newtrain_Y,
                                                         newtest_X, newtest_Y,
                                                         model_eval_train,
                                                         model_eval_test,
                                                         model_eval_train_y,
                                                         model_eval_test_y,
                                                         saveStep1,
                                                         Name=name_res2+'-'+name)
      saveStep1=0

        #print("classifier: ",name,"  - %s seconds -" % (time.time() - start_time))
      make_confusion_matrix_chart(cf_matrix_train, cf_matrix_test,name_res2+'-'+name)

print('#####################Train#####################')
df_train = pd.DataFrame.from_dict(model_eval_train)
df_train.to_csv(NewAdd+'Train.csv')

df_train1 = pd.DataFrame.from_dict(model_eval_train_y)
df_train1.to_json(NewAdd + 'LabelPredictTrain.json')

print('#####################Test#####################')
df_test = pd.DataFrame.from_dict(model_eval_test)
df_test.to_csv(NewAdd+'Test.csv')

df_test1 = pd.DataFrame.from_dict(model_eval_test_y)
df_test1.to_json(NewAdd + 'LabelPredictTest.json')



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def showROC(dfResult,NameFig,name_method,i_check):
  plt.figure()
  fig,axs=plt.subplots(1,2,figsize=(12,5))
  y_real=[]
  LegNameRoc=[]
  LegNamePRC=[]
  t=0
  if i_check==0: i_1,i_2=0,1
  if i_check==1: i_1,i_2=1,0
  for Name, Values in dfResult.items():
    if Name=='real':
      y_real=Values
      continue

    if (Name.split('-')[i_1]==name_method):
      y_pred=Values
      fpr, tpr, _ = roc_curve(y_real, y_pred)
      Precision1,recall1,_=precision_recall_curve(y_real, y_pred)
      f1score=f1_score(y_real, y_pred)
      roc_auc = auc(fpr, tpr)
      axs[0].plot(fpr, tpr)
      axs[1].plot(Precision1,recall1)
      LegNameRoc.append(Name.split('-')[i_2]+' ('+ str(roc_auc*100)[0:5]+' )')
      LegNamePRC.append(Name.split('-')[i_2]+' ('+ str(f1score*100)[0:5]+' )')
  axs[0].set_xlabel('False Positive Rate')
  axs[0].set_ylabel('True Positive Rate')
  axs[0].set_title('ROC Curve And AUC Of '+ name_method)
  axs[0].legend(LegNameRoc,loc='lower right')

  axs[1].set_xlabel('Recall')
  axs[1].set_ylabel('Precision')
  axs[1].set_title('Precision-Recall Curve And F1-Score Of '+ name_method)
  axs[1].legend(LegNamePRC,loc='lower left')
  plt.savefig(NewAdd+name_method+'-'+NameFig+'.png')
  plt.show()

#dfResult_Test=pd.read_csv(NewAdd+'LabelPredictTest.csv')
jsonResult_Test=pd.read_json(NewAdd+'LabelPredictTest.json')
jsonResult_Train=pd.read_json(NewAdd+'LabelPredictTrain.json')
for name_method_l1 in OLD_reSamplerName:
  for name_method_l2 in name_method_l1:
    showROC(jsonResult_Test.iloc[:,0],'Test',name_method_l2,0)
    showROC(jsonResult_Train.iloc[:,0],'Train',name_method_l2,0)

for name_method_l1 in names:
    showROC(jsonResult_Test.iloc[:,0],'Test',name_method_l1,1)
    showROC(jsonResult_Train.iloc[:,0],'Train',name_method_l1,1)


Output hidden; open in https://colab.research.google.com to view.